In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import json
import re
from shapely.geometry import Point, LineString #this library is for manipulating geometric objects, and it is what geopandas uses to store geometries
from scipy.spatial import distance

In [2]:
df = pd.read_csv('dataset_new.csv')

In [3]:
df.columns

Index(['Start.date', 'Start.station', 'End.date', 'End.station', 'id', 'lon',
       'lat'],
      dtype='object')

In [4]:
#covert the listing to the geo dataframe
g_df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs='epsg:27700')
g_df.crs

<Derived Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.0, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

In [5]:
g_df.head()

,Start.date,Start.station,End.date,End.station,id,lon,lat,geometry
0,2023/3/31 23:57,"Vauxhall Cross, Vauxhall",2023/3/31 23:58,"Kennington Lane Rail Bridge, Vauxhall",BikePoints_74,-0.124469,51.485917,POINT (-0.124 51.486)
1,2023/3/31 23:54,"Ossulston Street, Somers Town",2023/3/31 23:56,"Doric Way , Somers Town",BikePoints_797,-0.128279,51.529300,POINT (-0.128 51.529)
2,2023/3/31 23:54,"Parson's Green , Parson's Green",2023/3/31 23:58,"Elysium Place, Fulham",BikePoints_596,-0.199783,51.472817,POINT (-0.200 51.473)
3,2023/3/31 23:52,"Westfield Ariel Way, White City",2023/3/31 23:59,"St. Mark's Road, North Kensington",BikePoints_566,-0.224103,51.509158,POINT (-0.224 51.509)
4,2023/3/31 23:52,"Berry Street, Clerkenwell",2023/3/31 23:59,"Queen Street 1, Bank",BikePoints_246,-0.099994,51.522853,POINT (-0.100 51.523)


In [6]:
# Convert the Start.date and End.date to datetime format
g_df['Start.date'] = pd.to_datetime(g_df['Start.date'])
g_df['End.date'] = pd.to_datetime(g_df['End.date'])

# Set the time period
start_time = "16:00:00"
end_time = "19:00:00"

In [7]:
# Filter the dataset for rides between 07:00 and 09:00 during the period 17-23 October 2022
filtered_df = g_df[(g_df['Start.date'].dt.time >= pd.to_datetime(start_time).time()) &
                     (g_df['End.date'].dt.time <= pd.to_datetime(end_time).time()) &
                     (g_df['Start.date'].dt.date >= pd.to_datetime("2023-03-01").date()) &
                     (g_df['End.date'].dt.date <= pd.to_datetime("2023-03-31").date())]

In [8]:
filtered_df

,Start.date,Start.station,End.date,End.station,id,lon,lat,geometry
1643,2023-03-31 18:58:00,"Crawford Street, Marylebone",2023-03-31 18:59:00,"Crawford Street, Marylebone",BikePoints_43,-0.157183,51.520260,POINT (-0.157 51.520)
1649,2023-03-31 18:57:00,"Salmon Lane, Limehouse",2023-03-31 18:59:00,"Flamborough Street, Limehouse",BikePoints_542,-0.033828,51.514115,POINT (-0.034 51.514)
1666,2023-03-31 18:56:00,"St. Katharine's Way, Tower",2023-03-31 19:00:00,"Leman Street, Aldgate",BikePoints_452,-0.070542,51.505697,POINT (-0.071 51.506)
1673,2023-03-31 18:55:00,"Liverpool Road (N1 Centre), Angel",2023-03-31 18:59:00,"Finsbury Library , Finsbury",BikePoints_234,-0.106992,51.534504,POINT (-0.107 51.535)
1683,2023-03-31 18:54:00,"Cadogan Gardens, Chelsea",2023-03-31 18:58:00,"Chelsea Green, Chelsea",BikePoints_395,-0.159919,51.492462,POINT (-0.160 51.492)
...,...,...,...,...,...,...,...,...
563842,2023-03-20 16:00:00,"Houghton Street, Strand",2023-03-20 16:08:00,"Concert Hall Approach 2, South Bank",BikePoints_256,-0.116764,51.513620,POINT (-0.117 51.514)
563843,2023-03-20 16:00:00,"Breams Buildings, Holborn",2023-03-20 16:29:00,"Cephas Street, Bethnal Green",BikePoints_84,-0.111778,51.515937,POINT (-0.112 51.516)
563844,2023-03-20 16:01:00,"Shouldham Street, Marylebone",2023-03-20 16:05:00,"Hinde Street, Marylebone",BikePoints_396,-0.163609,51.518090,POINT (-0.164 51.518)
563853,2023-03-20 16:00:00,"Lambeth Palace Road, Waterloo",2023-03-20 16:04:00,"Sea Containers, South Bank",BikePoints_815,-0.116628,51.500089,POINT (-0.117 51.500)


In [9]:
columns = ['Start.station','End.station','geometry']
filtered_df= filtered_df[columns]
filtered_df

,Start.station,End.station,geometry
1643,"Crawford Street, Marylebone","Crawford Street, Marylebone",POINT (-0.157 51.520)
1649,"Salmon Lane, Limehouse","Flamborough Street, Limehouse",POINT (-0.034 51.514)
1666,"St. Katharine's Way, Tower","Leman Street, Aldgate",POINT (-0.071 51.506)
1673,"Liverpool Road (N1 Centre), Angel","Finsbury Library , Finsbury",POINT (-0.107 51.535)
1683,"Cadogan Gardens, Chelsea","Chelsea Green, Chelsea",POINT (-0.160 51.492)
...,...,...,...
563842,"Houghton Street, Strand","Concert Hall Approach 2, South Bank",POINT (-0.117 51.514)
563843,"Breams Buildings, Holborn","Cephas Street, Bethnal Green",POINT (-0.112 51.516)
563844,"Shouldham Street, Marylebone","Hinde Street, Marylebone",POINT (-0.164 51.518)
563853,"Lambeth Palace Road, Waterloo","Sea Containers, South Bank",POINT (-0.117 51.500)


In [10]:
# Suppose filtered_df is a filtered version of the original dataframe df
filtered_df1 = filtered_df.copy()

# Now you can modify filtered_df without warnings
filtered_df1['Route'] = filtered_df1['Start.station'] + ' - ' + filtered_df1['End.station']
df_route_counts = filtered_df1.groupby('Route').size().reset_index(name='Counts')
df_route_counts = df_route_counts.sort_values('Counts', ascending=False)

In [11]:
df_route_counts[['Start.station', 'End.station']] = df_route_counts['Route'].str.split(' - ', expand=True)

In [12]:
df_route_counts

,Route,Counts,Start.station,End.station
30212,"Hyde Park Corner, Hyde Park - Hyde Park Corner...",210,"Hyde Park Corner, Hyde Park","Hyde Park Corner, Hyde Park"
39077,"Newgate Street , St. Paul's - Waterloo Station...",133,"Newgate Street , St. Paul's","Waterloo Station 3, Waterloo"
64227,"Wormwood Street, Liverpool Street - Waterloo S...",104,"Wormwood Street, Liverpool Street","Waterloo Station 3, Waterloo"
45162,"Queen Street 1, Bank - Waterloo Station 3, Wat...",90,"Queen Street 1, Bank","Waterloo Station 3, Waterloo"
6237,"Black Lion Gate, Kensington Gardens - Black Li...",90,"Black Lion Gate, Kensington Gardens","Black Lion Gate, Kensington Gardens"
...,...,...,...,...
30752,"Import Dock, Canary Wharf - Graham Street, Angel",1,"Import Dock, Canary Wharf","Graham Street, Angel"
30754,"Import Dock, Canary Wharf - Gunmakers Lane, Ol...",1,"Import Dock, Canary Wharf","Gunmakers Lane, Old Ford"
30756,"Import Dock, Canary Wharf - Hop Exchange, The ...",1,"Import Dock, Canary Wharf","Hop Exchange, The Borough"
30759,"Import Dock, Canary Wharf - Kings Gate House, ...",1,"Import Dock, Canary Wharf","Kings Gate House, Westminster"


In [13]:
df_route_counts.to_csv('output_nightpeak.csv', index=False)